In [ ]:
"""This code is modified after reviewr comments where the data were normalize 
   to fix the arrows in the quiver function 
  ~~ Mukesh Rai = 2021/08/06"""

from netCDF4 import Dataset
import wrf
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import cartopy.crs as crs
from wrf import (to_np, getvar,interplevel, smooth2d, get_basemap, latlon_coords, ALL_TIMES)
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import cartopy.feature as cfe
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import AnchoredText
import cartopy
from cartopy.io import shapereader as shpreader
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.gridspec as gridspec
import pygrib
import cartopy.feature
#from cartopy.mpl.ticker import LaongitudeFormatter, LatitudeFormatter
import sys
import cartopy.crs as crs
import shapely.geometry as sgeom
from copy import copy
from math import sqrt
import cmaps
import matplotlib.colors as colors
import iris.quickplot as qplt
import matplotlib as mpl
import pygrib
from matplotlib import rcParams

"""Control font"""
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['font.sans-serif'] = "Times New Roman"


#======== Load WRF data ======================================================
win = [Dataset("wrfout_d01_2017-01-01_00_00_00"),
      Dataset("wrfout_d01_2017-02-01_00_00_00"),
      Dataset("wrfout_d01_2017-12-01_00_00_00")]
spr = [Dataset("wrfout_d01_2017-03-01_00_00_00.nc"),
       Dataset("wrfout_d01_2017-04-01_00_00_00.nc"),
       Dataset("wrfout_d01_2017-05-01_00_00_00.nc")]
sum = [Dataset("wrfout_d01_2017-06-01_00_00_00"),
       Dataset("wrfout_d01_2017-07-01_00_00_00"),
       Dataset("wrfout_d01_2017-08-01_00_00_00")]
aut = [Dataset("wrfout_d01_2017-09-01_00_00_00"),
       Dataset("wrfout_d01_2017-10-01_00_00_00"),
       Dataset("wrfout_d01_2017-11-01_00_00_00")]
#======= Load ERA5 =========================================
grbs = pygrib.open('jan_new.grib')
feb = pygrib.open('feb_new.grib')
mar = pygrib.open('mar_new.grib')
apr = pygrib.open('apr_new.grib')
may = pygrib.open('may_new.grib')
jun = pygrib.open('jun_new.grib')
jul = pygrib.open('jul_new.grib')
aug = pygrib.open('aug_new.grib')
sep = pygrib.open('sep_new.grib')
oct = pygrib.open('oct_new.grib')
nov = pygrib.open('nov_new.grib')
dec = pygrib.open('dec_new.grib')


#====== Get WRF data =====================================
p  = getvar(win, "pressure")
u1 = getvar(win, "ua", units="m s-1",timeidx=ALL_TIMES)
v1 = getvar(win, "va", units="m s-1",timeidx=ALL_TIMES)
u2 = getvar(spr, "ua", units="m s-1",timeidx=ALL_TIMES)
v2 = getvar(spr, "va", units="m s-1",timeidx=ALL_TIMES)
u3 = getvar(sum, "ua", units="m s-1",timeidx=ALL_TIMES)
v3 = getvar(sum, "va", units="m s-1",timeidx=ALL_TIMES)
u4 = getvar(aut, "ua", units="m s-1",timeidx=ALL_TIMES)
v4 = getvar(aut, "va", units="m s-1",timeidx=ALL_TIMES)

#=== Now interpolate at 500 hpa  ==============================
winu1 = interplevel(u1, p, 500)
winv1 = interplevel(v1, p, 500)
spru1 = interplevel(u2, p, 500)
sprv1 = interplevel(v2, p, 500)
sumu1 = interplevel(u3, p, 500)
sumv1 = interplevel(v3, p, 500)
autu1 = interplevel(u4, p, 500)
autv1 = interplevel(v4, p, 500)
#===== Take time average ===========================
winu = np.average(winu1,axis=0)
winv = np.average(winv1,axis=0)
spru = np.average(spru1,axis=0)
sprv = np.average(sprv1,axis=0)
sumu = np.average(sumu1,axis=0)
sumv = np.average(sumv1,axis=0)
autu = np.average(autu1,axis=0)
autv = np.average(autv1,axis=0)

#===== calculate seasonal wind speed =============
wi_ws1 = np.sqrt(winu**2+winv**2)
sp_ws1 = np.sqrt(spru**2+sprv**2)
su_ws1 = np.sqrt(sumu**2+sumv**2)
au_ws1 = np.sqrt(autu**2+autv**2)

#==== Normalize the data in uniform arrows ============
win_u_wrf = winu/wi_ws1
win_v_wrf = winv/wi_ws1
spr_u_wrf = spru/sp_ws1
spr_v_wrf = sprv/sp_ws1
sum_u_wrf = sumu/su_ws1
sum_v_wrf = sumv/su_ws1
aut_u_wrf = autu/au_ws1
aut_v_wrf = autv/au_ws1

#======== Get ERA5 data ====================================
for g in grbs:
    lat2, lon2 = g.latlons() # lats - latitude, lons - longitude
    lat2, lon2 = np.array(lat2), np.array(lon2)
#==============================================
jan_uwind = grbs.select()[0].values #U 10m wind component
jan_vwind = grbs.select()[1].values #V 10m wind component
feb_uwind = feb.select()[0].values
feb_vwind = feb.select()[1].values
mar_uwind = mar.select()[0].values
mar_vwind = mar.select()[1].values
apr_uwind = apr.select()[0].values
apr_vwind = apr.select()[1].values
may_uwind = may.select()[0].values
may_vwind = may.select()[1].values
jun_uwind = jun.select()[0].values
jun_vwind = jun.select()[1].values
jul_uwind = jul.select()[0].values
jul_vwind = jul.select()[1].values
aug_uwind = aug.select()[0].values
aug_vwind = aug.select()[1].values
sep_uwind = sep.select()[0].values
sep_vwind = sep.select()[1].values
oct_uwind = oct.select()[0].values
oct_vwind = oct.select()[1].values
nov_uwind = nov.select()[0].values
nov_vwind = nov.select()[1].values
dec_uwind = dec.select()[0].values
dec_vwind = dec.select()[1].values
#========= Seasonal ERA5 wind =========================================
wi_u = (jan_uwind+feb_uwind+dec_uwind)/3
wi_v = (jan_vwind+feb_vwind+dec_vwind)/3
sp_u = (mar_uwind+apr_uwind+may_uwind)/3
sp_v = (mar_vwind+apr_vwind+may_vwind)/3
su_u = (jun_uwind+jul_uwind+aug_uwind)/3
su_v = (jun_vwind+jul_vwind+aug_vwind)/3
au_u = (sep_uwind+oct_uwind+nov_uwind)/3
au_v = (sep_vwind+oct_vwind+nov_vwind)/3

#===== calculate seasonal wind speed =============
wi_ws2 = np.sqrt(wi_u**2+wi_v**2)
sp_ws2 = np.sqrt(sp_u**2+sp_v**2)
su_ws2 = np.sqrt(su_u**2+su_v**2)
au_ws2 = np.sqrt(au_u**2+au_v**2)
#==== Normalize the data in uniform arrows ============
wi_u_era = wi_u/wi_ws2
wi_v_era = wi_v/wi_ws2
sp_u_era = sp_u/sp_ws2
sp_v_era = sp_v/sp_ws2
su_u_era = su_u/su_ws2
su_v_era = su_v/su_ws2
au_u_era = au_u/au_ws2
au_v_era = au_v/au_ws2


#========== Get WRF lat lon  ===================
lat1, lon1 = latlon_coords(p)
#========== Get ERA5 lat lon  ====================================
for g in grbs:
    lat2, lon2 = g.latlons() # lats - latitude, lons - longitude
    lat2, lon2 = np.array(lat2), np.array(lon2)

"""
#===============================================================#
#++++++++++++++++++++++ Now Plot +++++++++++++++++++++++++++++++#
===============================================================#"""

fig = plt.figure(figsize=(6.2,2.6),dpi=300)

"""======= winter WRF ==================================================================="""
ax = plt.subplot(2, 4,1)
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(to_np(lon1), to_np(lat1)) # Convert the lat/lon coordinates to x/y coordinates in the projection space
#===== Plot windspeed in backround ===================================
q1 = plt.pcolormesh(x,y,wi_ws1,cmap=cmaps.cmocean_curl) #cmocean_curl)
plt.clim(0,20)
#==== Plot guiver ===========================================
q2 = plt.quiver(x[::5,::5], y[::5,::5], win_u_wrf[::5,::5], win_v_wrf[::5,::5],
              pivot='middle',scale_units='inches',headwidth=5,headlength=8)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.3, dashes=[2,2],  color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.3, dashes=[2, 2],  color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries(linewidth=0.3)
m.drawcoastlines(linewidth=0.3)
plt.ylabel('WRF', labelpad=5,fontsize=5)
plt.title('Winter',fontsize=5)
"""======= Spring WRF ==================================================================="""
ax = plt.subplot(2, 4,2)
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(to_np(lon1), to_np(lat1)) # Convert the lat/lon coordinates to x/y coordinates in the projection space
#===== Plot windspeed in backround ===================================
q1 = plt.pcolormesh(x,y,sp_ws1,cmap=cmaps.cmocean_curl) #cmocean_curl)
plt.clim(0,20)
#==== Plot guiver ===========================================
q2 = plt.quiver(x[::5,::5], y[::5,::5], spr_u_wrf[::5,::5], spr_v_wrf[::5,::5],
              pivot='middle',scale_units='inches',headwidth=5,headlength=8)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.3, dashes=[2, 2], color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.3, dashes=[2, 2],  color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries(linewidth=0.3)
m.drawcoastlines(linewidth=0.3)
plt.title('Spring',fontsize=5)

"""======= Summer WRF ==================================================================="""
ax = plt.subplot(2, 4,3)
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(to_np(lon1), to_np(lat1)) # Convert the lat/lon coordinates to x/y coordinates in the projection space
#===== Plot windspeed in backround ===================================
q1 = plt.pcolormesh(x,y,su_ws1,cmap=cmaps.cmocean_curl) #cmocean_curl)
plt.clim(0,20)
#==== Plot guiver ===========================================
q2 = plt.quiver(x[::5,::5], y[::5,::5], sum_u_wrf[::5,::5], sum_v_wrf[::5,::5],
              pivot='middle',scale_units='inches',headwidth=5,headlength=8)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.3, dashes=[2, 2], color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.3, dashes=[2, 2],  color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries(linewidth=0.3)
m.drawcoastlines(linewidth=0.3)
plt.title('Summer',fontsize=5)
"""======= Autumn WRF ==================================================================="""
ax = plt.subplot(2, 4,4)
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(to_np(lon1), to_np(lat1)) # Convert the lat/lon coordinates to x/y coordinates in the projection space
#===== Plot windspeed in backround ===================================
q1 = plt.pcolormesh(x,y,au_ws1,cmap=cmaps.cmocean_curl) #cmocean_curl)
plt.clim(0,20)
#==== Plot guiver ===========================================
q2 = plt.quiver(x[::5,::5], y[::5,::5], aut_u_wrf[::5,::5], aut_v_wrf[::5,::5],
              pivot='middle',scale_units='inches',headwidth=5,headlength=8)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.3, dashes=[2, 2], labels=[0,1,0,0], fontsize=4,color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.3,  dashes=[2, 2],  color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries(linewidth=0.3)
m.drawcoastlines(linewidth=0.3)
plt.title('Autumn',fontsize=5)
"""======= winter era5 ==================================================================="""
ax = plt.subplot(2, 4,5)
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(to_np(lon2), to_np(lat2)) # Convert the lat/lon coordinates to x/y coordinates in the projection space
#===== Plot windspeed in backround ===================================
q1 = plt.pcolormesh(x,y,wi_ws2,cmap=cmaps.cmocean_curl) #cmocean_curl)
plt.clim(0,20)
#==== Plot guiver ===========================================
q2 = plt.quiver(x[::7,::7], y[::7,::7], wi_u_era[::7,::7], wi_v_era[::7,::7],
              pivot='middle',scale_units='inches',headwidth=5,headlength=8)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.3, dashes=[2,2],  color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.3, dashes=[2, 2],   labels=[0,0,0,1], fontsize=4,color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries(linewidth=0.3)
m.drawcoastlines(linewidth=0.3)
plt.ylabel('ERA5', labelpad=5,fontsize=5)

"""======= spring era5 ==================================================================="""
ax = plt.subplot(2, 4,6)
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(to_np(lon2), to_np(lat2)) # Convert the lat/lon coordinates to x/y coordinates in the projection space
#===== Plot windspeed in backround ===================================
q1 = plt.pcolormesh(x,y,sp_ws2,cmap=cmaps.cmocean_curl) #cmocean_curl)
plt.clim(0,20)
#==== Plot guiver ===========================================
q2 = plt.quiver(x[::7,::7], y[::7,::7], sp_u_era[::7,::7], sp_v_era[::7,::7],
              pivot='middle',scale_units='inches',headwidth=5,headlength=8)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.3, dashes=[2,2],  color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.3, dashes=[2, 2],   labels=[0,0,0,1], fontsize=4,color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries(linewidth=0.3)
m.drawcoastlines(linewidth=0.3)


"""======= summer era5 ==================================================================="""
ax = plt.subplot(2, 4,7)
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(to_np(lon2), to_np(lat2)) # Convert the lat/lon coordinates to x/y coordinates in the projection space
#===== Plot windspeed in backround ===================================
q1 = plt.pcolormesh(x,y,su_ws2,cmap=cmaps.cmocean_curl) #cmocean_curl)
plt.clim(0,20)
#==== Plot guiver ===========================================
q2 = plt.quiver(x[::7,::7], y[::7,::7], su_u_era[::7,::7], su_v_era[::7,::7],
              pivot='middle',scale_units='inches',headwidth=5,headlength=8)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.3, dashes=[2,2],  color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.3, dashes=[2, 2],   labels=[0,0,0,1], fontsize=4,color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries(linewidth=0.3)
m.drawcoastlines(linewidth=0.3)

"""======= autum era5 ==================================================================="""
ax = plt.subplot(2, 4,8)
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(to_np(lon2), to_np(lat2)) # Convert the lat/lon coordinates to x/y coordinates in the projection space
#===== Plot windspeed in backround ===================================
ax1 = plt.pcolormesh(x,y,au_ws2,cmap=cmaps.cmocean_curl) #cmocean_curl)
plt.clim(0,20)
#==== Plot guiver ===========================================
q2 = plt.quiver(x[::7,::7], y[::7,::7], au_u_era[::7,::7], au_v_era[::7,::7],
              pivot='middle',scale_units='inches',headwidth=5,headlength=8)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.3, dashes=[2,2],  labels=[0,1,0,0], fontsize=4,color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.3, dashes=[2, 2],  labels=[0,0,0,1], fontsize=4,color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries(linewidth=0.3)
m.drawcoastlines(linewidth=0.3)




#====== Adjust====================================
fig.subplots_adjust(top=0.957,
                        bottom=0.14,
                        left=0.035,
                        right=.97,
                        hspace=0.0,
                        wspace=0.12)

cax = fig.add_axes([0.06,0.097,0.89,0.02])   # left, bottom, width, and height
cb = fig.colorbar(ax1,  cax=cax, orientation='horizontal',ticks=[0,2,4,6,8,10,12,14,16,18,20])
cb.set_label(r'Wind Speed  [$m  s^{-1}$]',fontsize=5,labelpad=0)
cb.ax.tick_params(labelsize=4)

plt.savefig("/mnt/g/2nd_Paper/wind.png",dpi=300)

#plt.show()
